In [1]:
import numpy as np
import pandas as pd

import extract

## Resources from the [football-data.org](https://www.football-data.org/documentation/quickstart) API

- competitions
- matches
- players
- scorers
- teams

In [2]:
def get_key_from_dictionary(obj, key):
    if not isinstance(obj, dict):
        return obj
    return obj.get(key)


def get_scoreline(data):
    score_pens = data['penalties']
    score_ft = data['fullTime']
    home_goals = score_ft['homeTeam']
    away_goals = score_ft['awayTeam']
    if type(score_pens['homeTeam']) is int:
        home_pens = score_pens['homeTeam']
        away_pens = score_pens['awayTeam']
        return f"{home_goals}-{away_goals} ({home_pens}-{away_pens} pens)"
    return f"{home_goals}-{away_goals}"


def get_goals_scored_from_scoreline(scoreline, by):
    if type(scoreline) is str:
        scoreline = scoreline.replace(' ', '')[:3]
        if '-' not in scoreline:
            return None
        home_goals, away_goals = scoreline.split('-')
        if by == 'home':
            return home_goals
        elif by == 'away':
            return away_goals
    return None


def clean_scorelines(data):
    data['homeGoals'] = data['score'].apply(get_goals_scored_from_scoreline, by='home')
    data['awayGoals'] = data['score'].apply(get_goals_scored_from_scoreline, by='away')
    return data


def get_refereeing_team(data):
    referee_team = ""
    for data_obj in data:
        if data_obj['name']:            
            referee_team += str(data_obj['name'])
            referee_team += ', '
    num_chars = len(referee_team)
    return referee_team[:num_chars-2]


# def convert_to_integer(obj):
#     try:
#         obj = int(obj)
#     except ValueError:
#         pass
#     return obj

## competitions and matches

In [3]:
%%time
dictionary_response = extract.get_raw_data(api_endpoint="/v2/competitions/CL/matches")
print(dictionary_response.keys())

dict_keys(['count', 'filters', 'competition', 'matches'])
Wall time: 2.3 s


In [4]:
df = pd.DataFrame(data=dictionary_response['matches'])
df['competition'] = dictionary_response['competition']['name']

In [5]:
print(df.shape)
df.sample(3)

(205, 14)


,id,season,utcDate,status,matchday,stage,group,lastUpdated,odds,score,homeTeam,awayTeam,referees,competition
180,283583,"{'id': 495, 'startDate': '2019-06-25', 'endDat...",2019-12-11T17:55:00Z,FINISHED,6.0,GROUP_STAGE,Group C,2019-12-12T20:20:00Z,{'msg': 'Activate Odds-Package in User-Panel t...,"{'winner': 'AWAY_TEAM', 'duration': 'REGULAR',...","{'id': 755, 'name': 'GNK Dinamo Zagreb'}","{'id': 65, 'name': 'Manchester City FC'}","[{'id': 43843, 'name': 'Carlos del Cerro', 'na...",UEFA Champions League
109,283551,"{'id': 495, 'startDate': '2019-06-25', 'endDat...",2019-10-01T19:00:00Z,FINISHED,2.0,GROUP_STAGE,Group A,2019-10-02T04:20:01Z,{'msg': 'Activate Odds-Package in User-Panel t...,"{'winner': 'AWAY_TEAM', 'duration': 'REGULAR',...","{'id': 610, 'name': 'Galatasaray SK'}","{'id': 524, 'name': 'Paris Saint-Germain FC'}","[{'id': 43888, 'name': 'Szymon Marciniak', 'na...",UEFA Champions League
188,285538,"{'id': 495, 'startDate': '2019-06-25', 'endDat...",2020-02-18T20:00:00Z,FINISHED,NaN,ROUND_OF_16,None,2020-02-19T20:50:00Z,{'msg': 'Activate Odds-Package in User-Panel t...,"{'winner': 'HOME_TEAM', 'duration': 'REGULAR',...","{'id': 78, 'name': 'Club Atlético de Madrid'}","{'id': 64, 'name': 'Liverpool FC'}","[{'id': 43888, 'name': 'Szymon Marciniak', 'na...",UEFA Champions League


In [6]:
resource_competitions_cols = df.columns.tolist()

In [7]:
columns_to_get_name = ['competition', 'homeTeam', 'awayTeam']
columns_in_dataframe = df.columns.tolist()
for column in columns_to_get_name:
    if column in columns_in_dataframe:
        df[column] = df[column].apply(get_key_from_dictionary, key='name')

df['score'] = df['score'].apply(get_scoreline)
df = clean_scorelines(data=df)

df['referees'] = df['referees'].apply(get_refereeing_team)
df['utcDate'] = pd.to_datetime(arg=df['utcDate'])
df.drop(labels=['lastUpdated', 'season', 'odds'], axis=1, inplace=True)

column_order = [
    'id', 'utcDate', 'competition', 'matchday', 'homeTeam', 'homeGoals', 'awayGoals', 'awayTeam',
    'score', 'group', 'stage', 'referees', 'status'
]

columns_in_dataframe = df.columns.tolist()
for column in column_order:
    if column not in columns_in_dataframe:
        column_order.remove(column)

df = df.loc[:, column_order]

In [ ]:
# df[(df['stage'] == 'ROUND_OF_16') & (df['status'] == 'FINISHED')].sample(5)

# team = 'Real Madrid CF'
# df[(df['homeTeam'] == team) | (df['awayTeam'] == team)].sample(3)

In [8]:
print(df.shape)
df.sample(3)

(205, 13)


,id,utcDate,competition,matchday,homeTeam,homeGoals,awayGoals,awayTeam,score,group,stage,referees,status
44,278366,2019-07-24 18:45:00+00:00,UEFA Champions League,NaN,Dundalk FC,1,1,Qarabağ Ağdam FK,1-1,None,2ND_QUALIFYING_ROUND,"Bartosz Frankowski, Marcin Boniek, Jakub Winkl...",FINISHED
4,269481,2019-07-09 14:00:00+00:00,UEFA Champions League,NaN,FC Ararat-Armenia,2,1,AIK Fotboll,2-1,None,1ST_QUALIFYING_ROUND,"Duje Strukan, Hrvoje Radić, Mario Pepur, Marin...",FINISHED
121,283634,2019-10-02 19:00:00+00:00,UEFA Champions League,2.0,Lille OSC,1,2,Chelsea FC,1-2,Group H,GROUP_STAGE,"Aliaksei Kulbakov, Dmitry Zhuk, Oleg Maslyanko...",FINISHED


In [9]:
df[df['score'].str.contains('pens')]

,id,utcDate,competition,matchday,homeTeam,homeGoals,awayGoals,awayTeam,score,group,stage,referees,status
25,269502,2019-07-17 16:30:00+00:00,UEFA Champions League,NaN,Rīga FC,0,0,Dundalk FC,0-0 (4-5 pens),None,1ST_QUALIFYING_ROUND,"Dimitar Mečkarovski, Goce Petreski",FINISHED
32,269496,2019-07-17 18:15:00+00:00,UEFA Champions League,NaN,FK Sutjeska Nikšić,1,1,ŠK Slovan Bratislava,1-1 (3-2 pens),None,1ST_QUALIFYING_ROUND,"Horațiu Fesnic, Alexandru Cerei, Mihai Marica,...",FINISHED
72,279612,2019-08-13 18:00:00+00:00,UEFA Champions League,NaN,FC København,1,1,FK Crvena Zvezda,1-1 (6-7 pens),None,3RD_QUALIFYING_ROUND,"Gediminas Mažeika, Vytautas Šimkus, Vytenis Ka...",FINISHED


## players

In [10]:
%%time
dictionary_response = extract.get_raw_data(api_endpoint="/v2/players/320/matches") # ?status=FINISHED
print(dictionary_response.keys())

dict_keys(['count', 'filters', 'player', 'matches'])
Wall time: 1.9 s


In [11]:
df = pd.DataFrame(data=dictionary_response['matches'])
df['player'] = dictionary_response['player']['name']
df.shape

(88, 15)

In [12]:
df.sample(3)

,id,competition,season,utcDate,status,matchday,stage,group,lastUpdated,odds,score,homeTeam,awayTeam,referees,player
23,271508,"{'id': 2002, 'name': 'Bundesliga'}","{'id': 474, 'startDate': '2019-08-16', 'endDat...",2019-11-10T14:30:00Z,FINISHED,11.0,REGULAR_SEASON,Regular Season,2019-11-11T14:50:05Z,{'msg': 'Activate Odds-Package in User-Panel t...,"{'winner': 'AWAY_TEAM', 'duration': 'REGULAR',...","{'id': 11, 'name': 'VfL Wolfsburg'}","{'id': 3, 'name': 'Bayer 04 Leverkusen'}","[{'id': 57510, 'name': 'Manuel Gräfe', 'nation...",Nadiem Amiri
78,201066,"{'id': 2002, 'name': 'Bundesliga'}","{'id': 3, 'startDate': '2017-08-18', 'endDate'...",2017-11-05T14:30:00Z,FINISHED,11.0,REGULAR_SEASON,Regular Season,2018-06-22T09:02:54Z,{'msg': 'Activate Odds-Package in User-Panel t...,"{'winner': 'AWAY_TEAM', 'duration': 'REGULAR',...","{'id': 1, 'name': '1. FC Köln'}","{'id': 2, 'name': 'TSG 1899 Hoffenheim'}","[{'id': 8825, 'name': 'Deniz Aytekin', 'nation...",Nadiem Amiri
40,235962,"{'id': 2002, 'name': 'Bundesliga'}","{'id': 155, 'startDate': '2018-08-24', 'endDat...",2019-04-28T13:30:00Z,FINISHED,31.0,REGULAR_SEASON,Regular Season,2019-04-28T18:30:26Z,{'msg': 'Activate Odds-Package in User-Panel t...,"{'winner': 'AWAY_TEAM', 'duration': 'REGULAR',...","{'id': 2, 'name': 'TSG 1899 Hoffenheim'}","{'id': 11, 'name': 'VfL Wolfsburg'}","[{'id': 8825, 'name': 'Deniz Aytekin', 'nation...",Nadiem Amiri


In [13]:
resource_players_cols = df.columns.tolist()

In [14]:
print(set(resource_competitions_cols).difference(set(resource_players_cols)))

print(set(resource_players_cols).difference(set(resource_competitions_cols)))

set()
{'player'}


In [15]:
columns_to_get_name = ['competition', 'homeTeam', 'awayTeam']
columns_in_dataframe = df.columns.tolist()
for column in columns_to_get_name:
    if column in columns_in_dataframe:
        df[column] = df[column].apply(get_key_from_dictionary, key='name')

df['score'] = df['score'].apply(get_scoreline)
df = clean_scorelines(data=df)

df['referees'] = df['referees'].apply(get_refereeing_team)
df['utcDate'] = pd.to_datetime(arg=df['utcDate'])
df.drop(labels=['lastUpdated', 'season', 'odds'], axis=1, inplace=True)

column_order = [
    'id', 'player', 'utcDate', 'competition', 'matchday', 'homeTeam', 'homeGoals', 'awayGoals', 'awayTeam',
    'score', 'group', 'stage', 'referees', 'status'
]

columns_in_dataframe = df.columns.tolist()
for column in column_order:
    if column not in columns_in_dataframe:
        column_order.remove(column)

df = df.loc[:, column_order]

In [16]:
print(df.shape)
df.sample(3)

(88, 14)


,id,player,utcDate,competition,matchday,homeTeam,homeGoals,awayGoals,awayTeam,score,group,stage,referees,status
25,271497,Nadiem Amiri,2019-11-02 14:30:00+00:00,Bundesliga,10.0,Bayer 04 Leverkusen,1,2,Borussia Mönchengladbach,1-2,Regular Season,REGULAR_SEASON,"Daniel Siebert, Lasse Koslowski, Jan Seidel, C...",FINISHED
74,201102,Nadiem Amiri,2017-12-10 14:30:00+00:00,Bundesliga,15.0,Hannover 96,2,0,TSG 1899 Hoffenheim,2-0,Regular Season,REGULAR_SEASON,"Markus Schmidt, Thorben Siewer, Daniel Schlage...",FINISHED
52,235854,Nadiem Amiri,2019-01-26 14:30:00+00:00,Bundesliga,19.0,SC Freiburg,2,4,TSG 1899 Hoffenheim,2-4,Regular Season,REGULAR_SEASON,"Bibiana Steinhaus, Guido Kleve, Thomas Stein, ...",FINISHED


## teams

In [17]:
%%time
dictionary_response = extract.get_raw_data(api_endpoint="/v2/teams/5")
print(dictionary_response.keys())

dict_keys(['id', 'area', 'activeCompetitions', 'name', 'shortName', 'tla', 'crestUrl', 'address', 'phone', 'website', 'email', 'founded', 'clubColors', 'venue', 'squad', 'lastUpdated'])
Wall time: 761 ms


In [18]:
df_by_team = pd.DataFrame(data=dictionary_response['squad'])

In [19]:
df_by_team.sample(3)

,id,name,position,dateOfBirth,countryOfBirth,nationality,shirtNumber,role
28,6543,Jann-Fiete Arp,Attacker,2000-01-06T00:00:00Z,Germany,Germany,NaN,PLAYER
1,342,Sven Ulreich,Goalkeeper,1988-08-03T00:00:00Z,Germany,Germany,NaN,PLAYER
27,3184,Leroy Sané,Attacker,1996-01-11T00:00:00Z,Germany,Germany,19.0,PLAYER


## scorers

In [20]:
%%time
dictionary_response = extract.get_raw_data(api_endpoint="/v2/competitions/CL/scorers?limit=40")
print(dictionary_response.keys())

dict_keys(['count', 'filters', 'competition', 'season', 'scorers'])
Wall time: 1.55 s


In [21]:
df_top_scorers = pd.DataFrame(data=dictionary_response['scorers'])

In [22]:
df_top_scorers.head(3)

,player,team,numberOfGoals
0,"{'id': 371, 'name': 'Robert Lewandowski', 'fir...","{'id': 5, 'name': 'FC Bayern München'}",11
1,"{'id': 38101, 'name': 'Erling Haaland', 'first...","{'id': 4, 'name': 'BV Borussia 09 Dortmund'}",10
2,"{'id': 73401, 'name': 'Mislav Oršić', 'firstNa...","{'id': 755, 'name': 'GNK Dinamo Zagreb'}",7


In [23]:
df_top_scorers['playerId'] = df_top_scorers['player'].apply(get_key_from_dictionary, key='id')
df_top_scorers['playerName'] = df_top_scorers['player'].apply(get_key_from_dictionary, key='name')
df_top_scorers['playerDob'] = df_top_scorers['player'].apply(get_key_from_dictionary, key='dateOfBirth')
df_top_scorers['playerNationality'] = df_top_scorers['player'].apply(get_key_from_dictionary, key='nationality')
df_top_scorers['playerPosition'] = df_top_scorers['player'].apply(get_key_from_dictionary, key='position')

df_top_scorers['teamId'] = df_top_scorers['team'].apply(get_key_from_dictionary, key='id')
df_top_scorers['teamName'] = df_top_scorers['team'].apply(get_key_from_dictionary, key='name')

df_top_scorers['playerDob'] = pd.to_datetime(arg=df_top_scorers['playerDob'])
df_top_scorers.drop(labels=['player', 'team'], axis=1, inplace=True)

In [24]:
df_top_scorers.head(3)

,numberOfGoals,playerId,playerName,playerDob,playerNationality,playerPosition,teamId,teamName
0,11,371,Robert Lewandowski,1988-08-21,Poland,Attacker,5,FC Bayern München
1,10,38101,Erling Haaland,2000-07-21,Norway,Attacker,4,BV Borussia 09 Dortmund
2,7,73401,Mislav Oršić,1992-12-29,Croatia,Attacker,755,GNK Dinamo Zagreb


In [25]:
df_top_scorers.dtypes

numberOfGoals                 int64
playerId                      int64
playerName                   object
playerDob            datetime64[ns]
playerNationality            object
playerPosition               object
teamId                        int64
teamName                     object
dtype: object

In [26]:
df_top_scorers

,numberOfGoals,playerId,playerName,playerDob,playerNationality,playerPosition,teamId,teamName
0,11,371,Robert Lewandowski,1988-08-21,Poland,Attacker,5,FC Bayern München
1,10,38101,Erling Haaland,2000-07-21,Norway,Attacker,4,BV Borussia 09 Dortmund
2,7,73401,Mislav Oršić,1992-12-29,Croatia,Attacker,755,GNK Dinamo Zagreb
3,6,36746,Billel Omrani,1993-06-02,France,Attacker,5515,FC CFR 1907 Cluj
4,6,2103,Dries Mertens,1987-05-06,Belgium,Attacker,113,SSC Napoli
5,6,8004,Harry Kane,1993-07-28,England,Attacker,73,Tottenham Hotspur FC
6,6,311,Serge Gnabry,1995-07-14,Germany,Midfielder,5,FC Bayern München
7,5,16207,Alexander Søderlund,1987-08-03,Norway,Attacker,889,Rosenborg BK
8,5,15549,Anders Konradsen,1990-07-18,Norway,Midfielder,889,Rosenborg BK
9,5,2174,Bruno Petković,1994-09-16,Croatia,Attacker,755,GNK Dinamo Zagreb


# Testing